<a href="https://colab.research.google.com/github/LeonardHolter/Little-Neural-Net/blob/main/micrograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random


In [ ]:
class Value:

  def __init__(self,data, _children=(), _op = ""):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op



  def __repr__(self) -> str:
    return f"value(data: {self.data})"



  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self,other), "+")

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad

    out._backward = _backward


    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self,other), "*")

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad

    out._backward = _backward


    return out



  def __rmul__(self, other):
    return self * other


  def __radd__(self, other):
    return self + other



  def __truediv__(self, other):
    return self * other**-1


  def __neg__(self):
    return self * -1

  def __sub__(self, other):
    return self + (-other)



  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only int/float support"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out


  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out



  def tanh(self):
    x = self.data
    t = (math.exp(2*x)-1 ) / (math.exp(2*x) + 1)
    out = Value(t, (self, ), "tanh")

    def _backward():
      self.grad += (1-t**2) * out.grad

    out._backward = _backward


    return out



  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()


In [ ]:
class Neuron:

  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out

  def parameters(self):
    return self.w + [self.b]



class Layer:

  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]


  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs


  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]


class MLP:

  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]


  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x


  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]



In [ ]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets


n = MLP(3, [4, 4, 1])





In [ ]:
for k in range(200):

  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

  # backward pass
  for p in n.parameters():
    p.grad = 0.0
  loss.backward()

  # update
  for p in n.parameters():
    p.data += -0.05 * p.grad

  print(k, loss.data)


0 0.0010314478581344131
1 0.001028678637029197
2 0.001025923902163984
3 0.001023183541273081
4 0.0010204574432413863
5 0.0010177454980897485
6 0.0010150475969605119
7 0.001012363632103355
8 0.0010096934968612668
9 0.0010070370856567826
10 0.0010043942939783974
11 0.001001765018367185
12 0.0009991491564036801
13 0.0009965466066948377
14 0.000993957268861308
15 0.0009913810435248206
16 0.0009888178322957665
17 0.0009862675377610125
18 0.0009837300634718434
19 0.0009812053139321168
20 0.0009786931945865416
21 0.0009761936118091826
22 0.000973706472892126
23 0.0009712316860342747
24 0.0009687691603303306
25 0.000966318805759932
26 0.0009638805331769807
27 0.0009614542542990552
28 0.0009590398816970125
29 0.0009566373287847905
30 0.000954246509809249
31 0.0009518673398402492
32 0.0009494997347608261
33 0.0009471436112575126
34 0.0009447988868108259
35 0.0009424654796858393
36 0.0009401433089229201
37 0.0009378322943286051
38 0.0009355323564665496
39 0.0009332434166487401
40 0.00093096539692

In [ ]:
ypred = [n(x) for x in xs]
print([y.data for y in ypred])


[0.9908333465281877, -0.990443935385936, -0.9853442881648009, 0.9833301282119941]
